<a href="https://colab.research.google.com/github/konatax/bookmanager/blob/main/keras_%E4%B9%B3%E8%85%BA%E7%99%8C%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! pip install tensorflow

In [ ]:
#将谷歌云盘挂载到远程主机（GPU）
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/fine_turn/keras-cancer")
#这一步之后可直接运行py文件
#! python example.py

In [ ]:
!nvidia-smi

In [22]:
from tensorflow.keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import numpy as np
from keras.layers import Dense,Dropout,Flatten
from keras.models import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
def cal_recall_right(result,label):#result是每个测试样本[y_pred,label],label是0或1
  num = 1*sum([1 for [s1,s2] in result if s1 == s2 and s2 == label])
  #用于计算准确率 
  num1 = sum([1 for [s1,s2] in result if s1 == label])
  #用于计算召回率
  num2 = sum([1 for [s1,s2] in result if s2 == label])

  if num == 0:
    return 0,0
  return num/num1,num/num2

In [24]:
#分批读取数据 rescale=1./255 表示图像每个像素值除以255 归一化，防止数据溢出
image_generator = ImageDataGenerator(rescale=1./255)
im_height, im_width = 224, 224
#构建vgg16模型（迁移学习），使用别人训练好的参数
model_vgg16 = VGG16(weights='imagenet')
#打印各层参数情况
print(model_vgg16.summary())

553476096/553467096 [==============================] - 19s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                           

In [25]:
#VGG16模型最后五层不用（只用前面几层现有的参数）
out = model_vgg16.layers[-5].output
#后面自己设置（是还未训练的）
out = Flatten()(out)
#全连接神经网络
out = Dense(100, activation='relu')(out)
#用来防止模型过拟合
out = Dropout(0.1)(out)
out = Dense(50, activation='relu')(out)
out = Dropout(0.1)(out)
#输出概率
out = Dense(1,activation='sigmoid')(out)
tuneModel = Model(inputs=model_vgg16.input,outputs=out)
for layer in tuneModel.layers[:-5]:
  #该层不进行训练
  layer.trainable = False
print(tuneModel.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [28]:
#样本非常多时 128
batch_size = 20
train_dir = 'valid'
#分批加载数据，每次读取batch_size张图片
train_data_gen = image_generator.flow_from_directory(directory=train_dir,batch_size=batch_size,shuffle=True,
                          target_size=(im_height,im_width),
                          class_mode='sparse')
total_train = train_data_gen.n
print(total_train)
print('---------------------')
print(train_data_gen.class_indices) #良性 恶性

Found 76 images belonging to 2 classes.
76
---------------------
{'benign': 0, 'malignant': 1}


class_mode: “categorical”, “binary”, “sparse”, “input” 或 None 之一。默认：“categorical”。决定返回的标签数组的类型：
1、"categorical" 将是 2D one-hot 编码标签，
2、"binary" 将是 1D 二进制标签，“sparse” 将是 1D 整数标签，
3、"input" 将是与输入图像相同的图像（主要用于自动编码器）。
如果为 None，不返回标签（生成器将只产生批量的图像数据，对于 model.predict_generator(), model.evaluate_generator() 等很有用）。请注意，如果 class_mode 为 None，那么数据仍然需要驻留在 directory 的子目录中才能正常工作。
————————————————
版权声明：本文为CSDN博主「是猪哥不是诸葛」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/my_name_is_learn/article/details/109536273

In [29]:
#加载图像次数
steps_per_epoch = (total_train / batch_size)

In [30]:
#训练模型
tuneModel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#class_weight 使模型更加注重恶性 （样本不均衡的时候恶性数据少，应增大权重，这里模拟样本不均衡 良性样本看一遍，恶性样本看二十遍
tuneModel.fit_generator(train_data_gen,epochs=20,steps_per_epoch=steps_per_epoch,class_weight={1:1.0, 0:20.0})
tuneModel.save('cancer_model.h5')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/20
3/3 [==============================] - 30s 3s/step - loss: 6.4816 - accuracy: 0.5132
Epoch 2/20
3/3 [==============================] - 1s 234ms/step - loss: 3.5042 - accuracy: 0.5263
Epoch 3/20
3/3 [==============================] - 1s 233ms/step - loss: 3.1766 - accuracy: 0.6842
Epoch 4/20
3/3 [==============================] - 1s 236ms/step - loss: 0.5917 - accuracy: 0.8421
Epoch 5/20
3/3 [==============================] - 1s 236ms/step - loss: 0.6178 - accuracy: 0.8816
Epoch 6/20
3/3 [==============================] - 1s 266ms/step - loss: 0.3114 - accuracy: 0.9079
Epoch 7/20
3/3 [==============================] - 1s 239ms/step - loss: 0.3719 - accuracy: 0.9079
Epoch 8/20
3/3 [==============================] - 1s 235ms/step - loss: 0.3465 - accuracy: 0.9079
Epoch 9/20
3/3 [==============================] - 1s 233ms/step - loss: 0.1818 - accuracy: 0.9474
Epoch 10/20
3/3 [==============================] - 1s 234ms/step - loss: 0.3168 - accuracy: 0.9474
Epoch 11/20
3/3 [====

In [32]:
type(train_data_gen[0])

tuple

In [34]:
type(train_data_gen[0][0])

numpy.ndarray

In [35]:
train_data_gen[0][0].shape  #三通道

(20, 224, 224, 3)

In [47]:
test_data = image_generator.flow_from_directory(directory=train_dir,batch_size=36,shuffle=True,
                          target_size=(im_height,im_width),
                          class_mode='sparse')
count = 0
result = []
threshold = 0.9  #阈值
for s in test_data:
  count += 1
  #将元组拆包
  img, label = s
  #模型输出的概率 是二维数组
  pred_y = tuneModel.predict(img).tolist()[0][0]
  #根据概率和阈值确定最终类别
  pred_y=1 if pred_y>threshold else 0
  label = label.tolist()[0]
  result.append([pred_y,label])
  if count >= test_data.n:
    break
acc0,recall0 = cal_recall_right(result,0)
acc1,recall1 = cal_recall_right(result,1)
print(test_data.class_indices)
print('----------------------')
print('良性','准确率：{}'.format(acc1),'召回率：{}'.format(recall1))
print('恶性','准确率：{}'.format(acc0),'召回率：{}'.format(recall0))

Found 76 images belonging to 2 classes.
{'benign': 0, 'malignant': 1}
----------------------
良性 准确率：1.0 召回率：0.9736842105263158
恶性 准确率：0.9743589743589743 召回率：1.0


In [48]:
label

0.0

In [50]:
s[0].shape,s[1].shape
#36张图像，224*224;  36个标签

((36, 224, 224, 3), (36,))

In [53]:
y = tuneModel.predict(img)
y.shape #36个测试样本的预测值，二维数组

(36, 1)